<a href="https://colab.research.google.com/github/marcusvpimentel/Mestrado/blob/main/Ajuste_autom%C3%A1tico_de_hiperpar%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X.shape, y.shape

((442, 10), (442,))

In [2]:
from sklearn.model_selection import train_test_split

Xtr, Xte, ytr, yte = train_test_split(X, y)
Xtr.shape, ytr.shape, Xte.shape, yte.shape

((331, 10), (331,), (111, 10), (111,))

In [5]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

modeloKnn = KNeighborsRegressor()
scores = cross_validate(modeloKnn, Xtr, ytr, cv=RepeatedKFold(random_state=42),
                        scoring=make_scorer(mean_squared_error, squared=False))
np.mean(scores['test_score'])

58.63710080535559

In [7]:
from sklearn.model_selection import GridSearchCV

param = {'n_neighbors': [3, 5, 7]}

modeloGSCV = GridSearchCV(KNeighborsRegressor(), param, verbose=3,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))
modeloGSCV.fit(X, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...................n_neighbors=3;, score=-58.073 total time=   0.0s
[CV 2/5] END ...................n_neighbors=3;, score=-67.430 total time=   0.0s
[CV 3/5] END ...................n_neighbors=3;, score=-62.289 total time=   0.0s
[CV 4/5] END ...................n_neighbors=3;, score=-62.023 total time=   0.0s
[CV 5/5] END ...................n_neighbors=3;, score=-67.250 total time=   0.0s
[CV 1/5] END ...................n_neighbors=5;, score=-56.278 total time=   0.0s
[CV 2/5] END ...................n_neighbors=5;, score=-63.421 total time=   0.0s
[CV 3/5] END ...................n_neighbors=5;, score=-59.637 total time=   0.0s
[CV 4/5] END ...................n_neighbors=5;, score=-59.591 total time=   0.0s
[CV 5/5] END ...................n_neighbors=5;, score=-61.706 total time=   0.0s
[CV 1/5] END ...................n_neighbors=7;, score=-56.992 total time=   0.0s
[CV 2/5] END ...................n_neighbors=7;, s

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [3, 5, 7]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False),
             verbose=3)

In [8]:
modeloGSCV.cv_results_

{'mean_fit_time': array([0.00348077, 0.00126233, 0.00110273]),
 'std_fit_time': array([0.00282644, 0.00018395, 0.00021232]),
 'mean_score_time': array([0.00858164, 0.00213799, 0.00217104]),
 'std_score_time': array([0.00647357, 0.00055404, 0.00126993]),
 'param_n_neighbors': masked_array(data=[3, 5, 7],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}],
 'split0_test_score': array([-58.07341774, -56.2778101 , -56.99180879]),
 'split1_test_score': array([-67.43002106, -63.42066441, -62.93518794]),
 'split2_test_score': array([-62.28861222, -59.63695849, -59.57483095]),
 'split3_test_score': array([-62.02315061, -59.59086644, -55.66109057]),
 'split4_test_score': array([-67.25015489, -61.70551029, -60.43146504]),
 'mean_test_score': array([-63.41307131, -60.12636195, -59.11887666]),
 'std_test_score': array([3.53743647, 2.3951518 , 2.56810112]),
 'rank_test_score': array([3

In [9]:
modeloGSCV.best_estimator_

KNeighborsRegressor(n_neighbors=7)

In [17]:
from math import sqrt
rmse = mean_squared_error(yte, modeloGSCV.predict(Xte), squared=False)
sqrt(rmse)
rmse

53.26908591182346

In [20]:
scores = cross_validate(modeloGSCV, X, y,
                        scoring=make_scorer(mean_squared_error, squared=False))
np.mean(scores['test_score'])


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...................n_neighbors=3;, score=-68.527 total time=   0.0s
[CV 2/5] END ...................n_neighbors=3;, score=-58.962 total time=   0.0s
[CV 3/5] END ...................n_neighbors=3;, score=-71.709 total time=   0.0s
[CV 4/5] END ...................n_neighbors=3;, score=-67.419 total time=   0.0s
[CV 5/5] END ...................n_neighbors=3;, score=-59.656 total time=   0.0s
[CV 1/5] END ...................n_neighbors=5;, score=-62.284 total time=   0.0s
[CV 2/5] END ...................n_neighbors=5;, score=-57.189 total time=   0.0s
[CV 3/5] END ...................n_neighbors=5;, score=-64.516 total time=   0.0s
[CV 4/5] END ...................n_neighbors=5;, score=-60.322 total time=   0.0s
[CV 5/5] END ...................n_neighbors=5;, score=-59.057 total time=   0.0s
[CV 1/5] END ...................n_neighbors=7;, score=-61.069 total time=   0.0s
[CV 2/5] END ...................n_neighbors=7;, s

59.1188766578109

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsRegressor())
])

scores = cross_validate(pipe, X, y,
                        scoring=make_scorer(mean_squared_error, squared=False))
rmse = np.mean(scores['test_score'])
print(f"KNNstd: {rmse}")

KNNstd: 60.45860455510717


In [24]:
modeloGSCV.verbose = 0
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', modeloGSCV)
  ])
scores = cross_validate(pipe, X, y,
                        scoring=make_scorer(mean_squared_error, squared=False))
rmse = np.mean(scores['test_score'])
print(f"GDSstd: {rmse}")

GDSstd: 58.858529150715086


In [25]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsRegressor())
])
param = {'scaler__with_mean': [True, False],
         'clf__n_neighbors': [3, 5, 7],
         'clf__metric': ['cosine', 'cityblock', 'euclidean']}
modeloAninhado = GridSearchCV(pipe, param, verbose=1, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))
scores = cross_validate(modeloAninhado, Xtr, ytr, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False))
np.mean(scores['test_score'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits


57.405950086546554

In [27]:
for modelo in scores['estimator']:
  print(modelo.best_params_)

{'clf__metric': 'cosine', 'clf__n_neighbors': 5, 'scaler__with_mean': False}
{'clf__metric': 'cosine', 'clf__n_neighbors': 7, 'scaler__with_mean': True}
{'clf__metric': 'cosine', 'clf__n_neighbors': 7, 'scaler__with_mean': False}
{'clf__metric': 'cosine', 'clf__n_neighbors': 7, 'scaler__with_mean': False}
{'clf__metric': 'cosine', 'clf__n_neighbors': 7, 'scaler__with_mean': False}
